# Extract areas from all available parameters and parse based on `source_id`

In [1]:
import intake
import numpy as np

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
from cmip6_preprocessing.parse_static_metrics import parse_metrics

In [3]:
# define collection
col = intake.open_esm_datastore("../../cmip6hack-ocean-bgc/catalogs/pangeo-cmip6.json")

In [4]:
# # load a bunch of dataset with intake_esm
# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     query = dict(experiment_id='piControl',
#                      variable_id=['thetao'], grid_label='gn')
#     cat = col.search(**query)
#     cat.df
#     raw_data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
#     raw_data_dict.keys()

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 22 group(s)


In [5]:
# from xgcm import Grid
# ds_test = raw_data_dict['CMIP.CNRM-CERFACS.CNRM-CM6-1.piControl.Omon.gn']
# grid = Grid(ds_test, coords={'X':{'center':'x'}})
# grid

<xgcm.Grid>
X Axis (periodic):
  * center   x

In [5]:
# raw_data_dict['CMIP.CNRM-CERFACS.CNRM-CM6-1.piControl.Omon.gn']

In [6]:
# # now parse all areas...
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     data_dict = parse_metrics(raw_data_dict, col)

You can see there are 22 models with temp data!

But not each one of them has an area.... It turns out that the areas are spread all over the catalogue with no discenible system. BUT THEY ARE SOMEWHERE for pretty much every model...and they should be the same for a given `source_id` and `grid_spec`. So with `parse_metrics` we can parse the area into the datasets as coordinates.

In [7]:
# new_dict = {k:ds for k, ds in data_dict.items() if 'areacello' in ds.coords}

In [8]:
# len(new_dict)

So we didnt get all of the models, but at least we got 17 to have an area for further calculations

In [9]:
# new_dict['CMIP.MIROC.MIROC6.piControl.Omon.gn'].thetao

In [10]:
query = dict(experiment_id='piControl',
                     variable_id=['thetao', 'uo', 'vo'],table_id='Omon', grid_label='gn')

In [11]:
# load the same thing with preprocessing
from cmip6_preprocessing.preprocessing import read_data
with warnings.catch_warnings(): # these lines just make sure that the warnings dont clutter your notebook
    warnings.simplefilter("ignore")
    data_dict = read_data(col, **query)

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 22 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 21 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 21 group(s)
BCC-CSM2-MR
Grid Type: B detected
BCC-ESM1
Grid Type: B detected
CAMS-CSM1-0
Grid Type: B detected
CanESM5
Grid Type: C detected
CNRM-CM6-1
Grid Type: C detected
<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lev: 75, member_id: 1, nvertex: 4, time: 6000, x: 362, x_left: 362, y: 294, y_left: 294)
Coordinates:
  * x            (x) int64 0 1 2 3 4 5 6 7 8 ... 354 355 356 357 358 359 360 361
  * y            (y) int64 0 1 2 3 4 5 6 7 8 ...

ValueError: Couldn't find a center coordinate for axis Y

In [ ]:
data_dict

In [ ]:
parse_metrics(data_dict, col, rename=True) #rename is important to get the consistent naming!